In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import doubletdetection
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = "Arial"
import scvelo as scv
scv.logging.print_version()
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualizationscv.settings.set_figure_params('scvelo') 
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))

Running scvelo 0.2.4 (python 3.9.7) on 2023-02-21 10:06.


ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.


In [8]:
adata_adult = sc.read('/public/home/zhaox/pigscRNA-seq/figures/figures/adult_spermatogoniaunannotation.h5ad') 
adata_adult

AnnData object with n_obs × n_vars = 757 × 4401
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [9]:
adata_adult.obs.cell_type

AAACCCAGTCCAGGTC         Diff SPG1
AAACCCAGTTCGGCGT    Early diff SPG
AAACGAACACCGTGCA    Early diff SPG
AAAGGATAGCGTGTCC         Diff SPG1
AAAGGATAGTCTCTGA    Early diff SPG
                         ...      
TTTCACACATAACGGG         Diff SPG1
TTTCAGTCACAATGAA         Diff SPG2
TTTGGAGAGGTAACTA         Diff SPG2
TTTGGAGTCCTGTACC    Early diff SPG
TTTGGTTGTGTCACAT         Diff SPG1
Name: cell_type, Length: 757, dtype: category
Categories (5, object): ['SSC1', 'SSC2', 'Early diff SPG', 'Diff SPG1', 'Diff SPG2']

In [15]:
adata_adult_ssc = adata_adult[adata_adult.obs['cell_type'].isin(['SSC1', 'SSC2'])].copy()
adata_adult_ssc

AnnData object with n_obs × n_vars = 222 × 4401
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [16]:
adata_adult_ssc.obs.cell_type

AAAGGATCAGCACAGA    SSC1
AAAGGGCGTATGCTAC    SSC1
AAAGTCCTCCACAAGT    SSC1
AACGAAAGTAACGGTG    SSC1
AACGAAATCCATACAG    SSC2
                    ... 
TTGTGTTAGCTCGCAC    SSC2
TTGTGTTCACTTGAAC    SSC1
TTGTTCAAGGCATCTT    SSC1
TTGTTCATCACCATAG    SSC2
TTGTTGTCAAAGTATG    SSC2
Name: cell_type, Length: 222, dtype: category
Categories (2, object): ['SSC1', 'SSC2']

In [17]:
adata_adult_ssc.obs.index=['adult_'+x for x in adata_adult_ssc.obs.index]

In [18]:
adata_adult_ssc.obs.index

Index(['adult_AAAGGATCAGCACAGA', 'adult_AAAGGGCGTATGCTAC',
       'adult_AAAGTCCTCCACAAGT', 'adult_AACGAAAGTAACGGTG',
       'adult_AACGAAATCCATACAG', 'adult_AAGGTAACAATCTCTT',
       'adult_AATAGAGAGACGGATC', 'adult_AATAGAGAGAGGTGCT',
       'adult_AATAGAGTCCGAGAAG', 'adult_AATCACGTCGGCATAT',
       ...
       'adult_TTCCACGAGACGAAGA', 'adult_TTCTAGTAGCCTGAAG',
       'adult_TTCTAGTCACGGTCTG', 'adult_TTGAACGGTACCGTCG',
       'adult_TTGCATTAGTCGAGGT', 'adult_TTGTGTTAGCTCGCAC',
       'adult_TTGTGTTCACTTGAAC', 'adult_TTGTTCAAGGCATCTT',
       'adult_TTGTTCATCACCATAG', 'adult_TTGTTGTCAAAGTATG'],
      dtype='object', length=222)

In [19]:
def map_condition(index):
    if 'adu' in index:
        return 'Adult'
    else:
        return 'Adult'

In [20]:
adata = adata_adult_ssc
index = adata.obs.index
index = ['neo_'+x for x in adata_adult_ssc.obs.index]
adata.obs['Sample'] = map_condition(index)

In [21]:
adata_adult_ssc.obs.Sample

adult_AAAGGATCAGCACAGA    Adult
adult_AAAGGGCGTATGCTAC    Adult
adult_AAAGTCCTCCACAAGT    Adult
adult_AACGAAAGTAACGGTG    Adult
adult_AACGAAATCCATACAG    Adult
                          ...  
adult_TTGTGTTAGCTCGCAC    Adult
adult_TTGTGTTCACTTGAAC    Adult
adult_TTGTTCAAGGCATCTT    Adult
adult_TTGTTCATCACCATAG    Adult
adult_TTGTTGTCAAAGTATG    Adult
Name: Sample, Length: 222, dtype: object

In [10]:
adata_neo = sc.read('/public/home/zhaox/pigscRNA-seq/figures/neospg_annotation.h5ad') 
adata_neo

AnnData object with n_obs × n_vars = 184 × 4944
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [11]:
adata_neo.obs.cell_type

AAAGGGCAGGCTAGCA-1         SSC
AACAAGACACCTGTCT-1         SSC
AACCACATCACTTCTA-1    PreSPG-1
AAGACTCCAACACTAC-1         SSC
AAGACTCCACTGCATA-1    PreSPG-1
                        ...   
TTCCGTGGTCGTTTCC-1    PreSPG-1
TTCTTCCCAGACTCTA-1         SSC
TTGCGTCGTACTTGTG-1         SSC
TTGGGCGGTCACTGAT-1         SSC
TTTAGTCAGCGTGCCT-1         SSC
Name: cell_type, Length: 184, dtype: category
Categories (3, object): ['PreSPG-1', 'PreSPG-2', 'SSC']

In [22]:
adata_neo.obs.index=['neo_'+x for x in adata_neo.obs.index]
adata_neo.obs.index

Index(['neo_AAAGGGCAGGCTAGCA-1', 'neo_AACAAGACACCTGTCT-1',
       'neo_AACCACATCACTTCTA-1', 'neo_AAGACTCCAACACTAC-1',
       'neo_AAGACTCCACTGCATA-1', 'neo_AAGCATCCAATCGCCG-1',
       'neo_AAGTTCGAGCACCGAA-1', 'neo_AATCGACTCCGACGGT-1',
       'neo_AATGCCAAGCAATTCC-1', 'neo_ACACCAATCCCGAGAC-1',
       ...
       'neo_TGTTCCGAGTGAGCCA-1', 'neo_TTAATCCGTTACTCAG-1',
       'neo_TTACAGGCAGCGAGTA-1', 'neo_TTACTGTCACCGGAAA-1',
       'neo_TTCACCGGTCATGCAT-1', 'neo_TTCCGTGGTCGTTTCC-1',
       'neo_TTCTTCCCAGACTCTA-1', 'neo_TTGCGTCGTACTTGTG-1',
       'neo_TTGGGCGGTCACTGAT-1', 'neo_TTTAGTCAGCGTGCCT-1'],
      dtype='object', length=184)

In [23]:
def map_condition(index):
    if 'neo' in index:
        return 'Neonatal'
    else:
        return 'Neonatal'

In [24]:
adata = adata_neo
index = adata.obs.index
index = ['neo_'+x for x in adata_neo.obs.index]
adata.obs['Sample'] = map_condition(index)

In [25]:
adata_neo.obs.Sample

neo_AAAGGGCAGGCTAGCA-1    Neonatal
neo_AACAAGACACCTGTCT-1    Neonatal
neo_AACCACATCACTTCTA-1    Neonatal
neo_AAGACTCCAACACTAC-1    Neonatal
neo_AAGACTCCACTGCATA-1    Neonatal
                            ...   
neo_TTCCGTGGTCGTTTCC-1    Neonatal
neo_TTCTTCCCAGACTCTA-1    Neonatal
neo_TTGCGTCGTACTTGTG-1    Neonatal
neo_TTGGGCGGTCACTGAT-1    Neonatal
neo_TTTAGTCAGCGTGCCT-1    Neonatal
Name: Sample, Length: 184, dtype: object

In [26]:
import anndata as ad
adatas = [adata_adult_ssc,adata_neo]
adata_ssc = ad.concat(adatas,merge="same")
adata_ssc

AnnData object with n_obs × n_vars = 406 × 1534
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'Sample'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'chrX', 'chrY', 'highly_variable'
    obsm: 'X_pca', 'X_tsne', 'X_umap'

In [27]:
adata_ssc.obs.Sample

adult_AAAGGATCAGCACAGA       Adult
adult_AAAGGGCGTATGCTAC       Adult
adult_AAAGTCCTCCACAAGT       Adult
adult_AACGAAAGTAACGGTG       Adult
adult_AACGAAATCCATACAG       Adult
                            ...   
neo_TTCCGTGGTCGTTTCC-1    Neonatal
neo_TTCTTCCCAGACTCTA-1    Neonatal
neo_TTGCGTCGTACTTGTG-1    Neonatal
neo_TTGGGCGGTCACTGAT-1    Neonatal
neo_TTTAGTCAGCGTGCCT-1    Neonatal
Name: Sample, Length: 406, dtype: object

In [28]:
results_file = './figures/adultssc_neogerm_unannotation0221.h5ad'
adata_ssc.write(results_file)

In [29]:
adata_ssc = adata_ssc.raw.to_adata()
adata_ssc

AnnData object with n_obs × n_vars = 406 × 18859
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'Sample'
    obsm: 'X_pca', 'X_tsne', 'X_umap'

In [30]:
adata_ssc.obs.cell_type

adult_AAAGGATCAGCACAGA        SSC1
adult_AAAGGGCGTATGCTAC        SSC1
adult_AAAGTCCTCCACAAGT        SSC1
adult_AACGAAAGTAACGGTG        SSC1
adult_AACGAAATCCATACAG        SSC2
                            ...   
neo_TTCCGTGGTCGTTTCC-1    PreSPG-1
neo_TTCTTCCCAGACTCTA-1         SSC
neo_TTGCGTCGTACTTGTG-1         SSC
neo_TTGGGCGGTCACTGAT-1         SSC
neo_TTTAGTCAGCGTGCCT-1         SSC
Name: cell_type, Length: 406, dtype: category
Categories (5, object): ['PreSPG-1', 'PreSPG-2', 'SSC', 'SSC1', 'SSC2']

In [31]:
sc.pl.umap(adata_ssc, color='cell_type', frameon=True, legend_loc= "right margin", legend_fontsize=10, legend_fontweight="normal", legend_fontoutline=2, save="adultsscneogerm_celltyperight0221.pdf")

In [32]:
%%time
sc.external.pp.bbknn(adata_ssc, batch_key='Sample')

2023-02-21 11:04:42.371930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /public/home/zhaox/anaconda3/envs/jupyter/lib:/opt/OpenBLAS/lib
2023-02-21 11:04:42.371983: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


CPU times: user 12.5 s, sys: 834 ms, total: 13.3 s
Wall time: 23.3 s


In [34]:
sc.tl.umap(adata_ssc)

In [35]:
sc.pl.umap(adata_ssc, color=['Sample', 'cell_type'], frameon=True, legend_loc= "right margin", legend_fontsize=10, legend_fontweight="normal", legend_fontoutline=2, save="adultsscneogerm_celltypebbknnright0221.pdf")

In [36]:
sc.tl.rank_genes_groups(adata_ssc, 'cell_type', method='wilcoxon')

In [39]:
from matplotlib.pyplot import rc_context
with rc_context({'figure.figsize': (9, 1.5)}):
    sc.pl.rank_genes_groups_violin(adata_ssc, n_genes=20, jitter=True,save="adultsscneogerm_0221.pdf")

In [45]:
markers = ['POU5F1','PIM2','ETV4','DOCK8','SERINC2','TTC14','COL1A2','TIMP2','DVL1','ZFHX3',
          'LY6K','UTF1','PIWIL4','LPPR3','GFRA1','UCHL1']
with rc_context({'figure.figsize': (4.5, 3)}):
    sc.pl.violin(adata_ssc, ['POU5F1','PIM2','ETV4','DOCK8','SERINC2'], groupby='cell_type',stripplot=False, inner='box',rotation=90,save="adultsscneogermvio1_0221.pdf")

In [46]:
with rc_context({'figure.figsize': (4.5, 3)}):
    sc.pl.violin(adata_ssc, ['TTC14','COL1A2','TIMP2','DVL1','ZFHX3'], groupby='cell_type',stripplot=False, inner='box',rotation=90,save="adultsscneogermvio2_0221.pdf")

In [48]:
with rc_context({'figure.figsize': (4.5, 3)}):
    sc.pl.violin(adata_ssc, ['LY6K','UTF1','PIWIL4','GFRA1','UCHL1'], groupby='cell_type',stripplot=False, inner='box',rotation=90,save="adultsscneogermvio3_0221.pdf")

In [49]:
cell_cycle_genes = [x.strip() for x in open('/public/home/zhaox/pigscRNA-seq/cell_cycle_genes (remove).txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

In [50]:
ax = sc.pl.heatmap(adata_ssc, s_genes, groupby='cell_type', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,3),save='g1scellcycleadultsscneogerm0221.pdf')
ax = sc.pl.heatmap(adata_ssc, g2m_genes, groupby='cell_type', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,4),save='g2mcellcycleadultsscneogerm0221.pdf')

In [51]:
pd.DataFrame(adata_ssc.uns['rank_genes_groups']['names']).head(5)
adata_ssc.uns['rank_genes_groups'].keys()
#dict_keys(['logfoldchanges', 'names', 'params', 'pvals', 'pvals_adj', 'scores'])
result = adata_ssc.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)
res = pd.DataFrame(    
    {group + '_' + key: result[key][group]    
    for group in groups for key in ['names', 'pvals','logfoldchanges','pvals_adj','scores']})
res.to_csv("./figures/diff_rank_genes_groups_adultsscneogermcelltype0221.csv")

In [55]:
sc.tl.rank_genes_groups(adata_ssc, 'cell_type', method='wilcoxon')
sc.pl.rank_genes_groups_heatmap(adata_ssc, use_raw=False,vmin=-3, vmax=3, min_logfoldchange=1,groupby='cell_type',n_genes=50, cmap='RdBu_r', show=False, save = "adultsscneogermuseraw0221.pdf")

{'heatmap_ax': <AxesSubplot: >,
 'groupby_ax': <AxesSubplot: ylabel='cell_type'>,
 'dendrogram_ax': <AxesSubplot: >,
 'gene_groups_ax': <AxesSubplot: >}

# 补充图片用

In [58]:
results_file = './adultsscneogerm_merged0221.h5ad'
adata_ssc.write(results_file)

In [57]:
adata_ssc

AnnData object with n_obs × n_vars = 406 × 18859
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'Sample'
    uns: 'cell_type_colors', 'neighbors', 'Sample_colors', 'umap', 'rank_genes_groups', 'dendrogram_cell_type'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'distances', 'connectivities'

In [60]:
adultsscneogerm_mergedforcellrank = adata_ssc
adultsscneogerm_mergedforcellrank

AnnData object with n_obs × n_vars = 406 × 18859
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'Sample'
    uns: 'cell_type_colors', 'neighbors', 'Sample_colors', 'umap', 'rank_genes_groups', 'dendrogram_cell_type'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'distances', 'connectivities'

In [61]:
import scvelo as scv
scv.logging.print_version()
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualizationscv.settings.set_figure_params('scvelo') 
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))

Running scvelo 0.2.4 (python 3.9.7) on 2023-02-21 15:31.


Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.


In [62]:
adultsscneogerm_mergedforcellrank.obs.Sample

adult_AAAGGATCAGCACAGA       Adult
adult_AAAGGGCGTATGCTAC       Adult
adult_AAAGTCCTCCACAAGT       Adult
adult_AACGAAAGTAACGGTG       Adult
adult_AACGAAATCCATACAG       Adult
                            ...   
neo_TTCCGTGGTCGTTTCC-1    Neonatal
neo_TTCTTCCCAGACTCTA-1    Neonatal
neo_TTGCGTCGTACTTGTG-1    Neonatal
neo_TTGGGCGGTCACTGAT-1    Neonatal
neo_TTTAGTCAGCGTGCCT-1    Neonatal
Name: Sample, Length: 406, dtype: category
Categories (2, object): ['Adult', 'Neonatal']

In [63]:
adata_adult = adultsscneogerm_mergedforcellrank[adultsscneogerm_mergedforcellrank.obs['Sample'].isin(['Adult'])].copy()
adata_adult

AnnData object with n_obs × n_vars = 222 × 18859
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'Sample'
    uns: 'cell_type_colors', 'neighbors', 'Sample_colors', 'umap', 'rank_genes_groups', 'dendrogram_cell_type'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'distances', 'connectivities'

In [64]:
adata_neo = adultsscneogerm_mergedforcellrank[adultsscneogerm_mergedforcellrank.obs['Sample'].isin(['Neonatal'])].copy()
adata_neo

AnnData object with n_obs × n_vars = 184 × 18859
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'Sample'
    uns: 'cell_type_colors', 'neighbors', 'Sample_colors', 'umap', 'rank_genes_groups', 'dendrogram_cell_type'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'distances', 'connectivities'

In [65]:
adataadult_loom = scv.read('/public/home/zhaox/pigscRNA-seq/Adult/Adult.loom', cache=True)
scv.pl.proportions(adataadult_loom,save="./figures/adultsscneogerm_merged0221forscvelo.pdf")
adata_adult = scv.utils.merge(adata_adult, adataadult_loom)
scv.pl.proportions(
    adata_adult,
    groupby='cell_type',
    layers=None,
    highlight='unspliced',
    add_labels_pie=True,
    add_labels_bar=True,
    fontsize=8,
    figsize=(15, 2),
    dpi=600,
    use_raw=True,
    show=True,
    save="./figures/adultsscneogerm_mergedforscvelospliced0221.pdf",
)

saving figure to file ./figures/adultsscneogerm_merged0221forscvelo.pdf
saving figure to file ./figures/adultsscneogerm_mergedforscvelospliced0221.pdf


In [66]:
adataneo_loom = scv.read('/public/home/zhaox/pigscRNA-seq/P5/P5.loom', cache=True)
scv.pl.proportions(adataneo_loom,save="./figures/adultsscneogerm_mergedneo0221forscvelo.pdf")
adata_neo = scv.utils.merge(adata_neo, adataneo_loom)
scv.pl.proportions(
    adata_neo,
    groupby='cell_type',
    layers=None,
    highlight='unspliced',
    add_labels_pie=True,
    add_labels_bar=True,
    fontsize=8,
    figsize=(15, 2),
    dpi=600,
    use_raw=True,
    show=True,
    save="./figures/adultsscneogerm_mergedneoforscvelospliced0221.pdf",
)

saving figure to file ./figures/adultsscneogerm_mergedneo0221forscvelo.pdf
saving figure to file ./figures/adultsscneogerm_mergedneoforscvelospliced0221.pdf


In [67]:
import anndata as ad
adatas = [adata_adult,adata_neo]
adata = ad.concat(adatas,merge="same")
adata

AnnData object with n_obs × n_vars = 406 × 18859
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'Sample', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [68]:
sc.pl.umap(adata, color=['Sample', 'cell_type'], frameon=True, legend_loc= "right margin", legend_fontsize=10, legend_fontweight="normal", legend_fontoutline=2, save="adultsscneogerm_celltypeforscveloright0221.pdf")

In [69]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

Filtered out 13594 genes that are detected 20 counts (shared).
Normalized count data: spliced, unspliced.
Extracted 2000 highly variable genes.
computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [70]:
adata

AnnData object with n_obs × n_vars = 406 × 2000
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'Sample', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'Sample_colors', 'cell_type_colors', 'neighbors'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced', 'Ms', 'Mu'
    obsp: 'distances', 'connectivities'

In [71]:
scv.tl.recover_dynamics(adata, n_jobs=8)
scv.tl.velocity(adata, mode="dynamical")
scv.tl.velocity_graph(adata)

recovering dynamics (using 8/24 cores)


  0%|          | 0/659 [00:00<?, ?gene/s]

    finished (0:00:16) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/24 cores)


  0%|          | 0/406 [00:00<?, ?cells/s]

    finished (0:00:00) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [72]:
adata.write('./adultsscneogermforscvelo.h5ad', compression='gzip')
adata = scv.read('./adultsscneogermforscvelo.h5ad')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap')


In [73]:
scv.pl.velocity_embedding_stream(
    adata, basis="umap",color='cell_type', legend_fontsize=12, title="", smooth=0.8, min_mass=4,save='adultsscneogermforscvelo_embeddingstream0221pdf'
)

computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
figure cannot be saved as pdf, using png instead.
saving figure to file ./figures/scvelo_adultsscneogermforscvelo_embeddingstream0221pdf.png


In [76]:
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot',save='adultsscneogermforscvelo_latenttime0221pdf')

computing latent time using root_cells as prior
    finished (0:00:00) --> added 
    'latent_time', shared time (adata.obs)
saving figure to file ./figures/scvelo_adultsscneogermforscvelo_latenttime0221pdf.pdf


In [77]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adata, var_names=top_genes, sortby='latent_time', col_color='cell_type', n_convolve=100,save='adultsscneogermforscvelo_topgenes0221pdf')

saving figure to file ./figures/scvelo_heatmap_adultsscneogermforscvelo_topgenes0221pdf.pdf


In [79]:
# Top-likelihood genes
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(adata, basis=top_genes[:15], ncols=5,color='cell_type', frameon=False,save='adultsscneogermforscvelo_topgenes15_0221pdf')

saving figure to file ./figures/scvelo_adultsscneogermforscvelo_topgenes15_0221pdf.pdf


In [80]:
scv.pl.scatter(adata, basis=top_genes[15:30], ncols=5,color='cell_type', frameon=False,save='adultsscneogermforscvelo_topgenes30_0221pdf')

saving figure to file ./figures/scvelo_adultsscneogermforscvelo_topgenes30_0221pdf.pdf


In [ ]:
markers = ['POU5F1','PIM2','ETV4','DOCK8','SERINC2','TTC14','COL1A2','TIMP2','DVL1','ZFHX3',
          'LY6K','UTF1','PIWIL4','LPPR3','GFRA1','UCHL1']

In [85]:
var_names = ['DOCK8','COL1A2','TIMP2','ZFHX3']
scv.pl.scatter(adata, var_names,color='cell_type', frameon=False,save='adultsscneogermforscvelo_DOCK8_0221pdf')
scv.pl.scatter(adata, x='latent_time', y=var_names,color='cell_type', frameon=False,save='adultsscneogermforscvelo_DOCK8latent_0221pdf')

saving figure to file ./figures/scvelo_adultsscneogermforscvelo_DOCK8_0221pdf.pdf
saving figure to file ./figures/scvelo_adultsscneogermforscvelo_DOCK8latent_0221pdf.pdf


In [90]:
var_names = ['PIWIL4','GFRA1','UCHL1','ETV5']
scv.pl.scatter(adata, var_names,color='cell_type', frameon=False,save='adultsscneogermforscvelo_PIWIL4_0221pdf')
scv.pl.scatter(adata, x='latent_time', y=var_names,color='cell_type', frameon=False,save='adultsscneogermforscvelo_PIWIL4latent_0221pdf')

saving figure to file ./figures/scvelo_adultsscneogermforscvelo_PIWIL4_0221pdf.pdf
saving figure to file ./figures/scvelo_adultsscneogermforscvelo_PIWIL4latent_0221pdf.pdf


# Cluster-specific top-likelihood genes

In [91]:
scv.tl.rank_dynamical_genes(adata, groupby='cell_type')
df = scv.get_df(adata, 'rank_dynamical_genes/names')
df.head(5)

ranking genes by cluster-specific likelihoods
    finished (0:00:00) --> added 
    'rank_dynamical_genes', sorted scores by group ids (adata.uns)


,PreSPG-1,PreSPG-2,SSC,SSC1,SSC2
0,RPS21,ZZZ3,RPS27,RPL23,DMRT1
1,RPL23,RYR2,LRP1B,RPS27,MCL1
2,ZC3H10,KLHL15,FTL,ZC3H10,LOC110257929
3,RPS27,RPS5,MYBPC1,FAU,ZZZ3
4,RPL28,RPS27,RPS21,RPL28,RYR2


In [94]:
for cluster in ['PreSPG-1', 'PreSPG-2', 'SSC', 'SSC1','SSC2']:
    scv.pl.scatter(adata, df[cluster][:5], ylabel=cluster,color='cell_type', frameon=False)

In [95]:
plt.savefig('adultsscneogermforscveloclustertop0221.pdf')

In [100]:
for cluster in ['SSC2']:
    scv.pl.scatter(adata, df[cluster][:15],ncols=5, ylabel=cluster,color='cell_type', frameon=False,save='adultsscneogermforscvelo_SSC2top0221.pdf')

saving figure to file ./figures/scvelo_adultsscneogermforscvelo_SSC2top0221.pdf


# Interprete the velocities 

In [103]:
adata = scv.read('./adultsscneogermforscvelo.h5ad')

In [107]:
scv.tl.velocity(adata)
scv.tl.velocity_graph(adata)

computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/24 cores)


  0%|          | 0/406 [00:00<?, ?cells/s]

    finished (0:00:01) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [111]:
scv.pl.velocity_embedding(adata, arrow_length=5, arrow_size=4, size=500,dpi=900,color='cell_type',save='adultsscneogermforscvelo_basic0221.pdf')

saving figure to file ./figures/scvelo_adultsscneogermforscvelo_basic0221.pdf


In [112]:
scv.pl.velocity(adata, ['PIWIL4','GFRA1','UCHL1','ETV5'], ncols=2,color='cell_type',save='adultsscneogermforscvelo_velocitiesPIWIL4basic0221.pdf')

saving figure to file ./figures/scvelo_adultsscneogermforscvelo_velocitiesPIWIL4basic0221.pdf


In [113]:
scv.pl.velocity(adata, ['DOCK8','COL1A2','TIMP2','ZFHX3'], ncols=2,color='cell_type',save='adultsscneogermforscvelo_velocitiesDOCK8basic0221.pdf')

saving figure to file ./figures/scvelo_adultsscneogermforscvelo_velocitiesDOCK8basic0221.pdf


## cellrank

In [114]:
adata = scv.read('./adultsscneogermforscvelo.h5ad')
scv.tl.recover_dynamics(adata, n_jobs=8)
scv.tl.velocity(adata, mode="dynamical")
scv.tl.velocity_graph(adata)

recovering dynamics (using 8/24 cores)


  0%|          | 0/612 [00:00<?, ?gene/s]

    finished (0:00:15) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/24 cores)


  0%|          | 0/406 [00:00<?, ?cells/s]

    finished (0:00:01) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [116]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2
import warnings
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

In [117]:
cr.tl.terminal_states(adata, cluster_key="cell_type", weight_connectivities=0.2)

Accessing `adata.obsp['T_fwd']`
Computing transition matrix based on logits using `'deterministic'` mode
Estimating `softmax_scale` using `'deterministic'` mode


/tmp/ipykernel_167116/637574554.py:1: DeprecationWarning: `cellrank.tl.terminal_states` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.terminal_states(adata, cluster_key="cell_type", weight_connectivities=0.2)
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/cellrank/tl/_init_term_states.py:156: DeprecationWarning: `cellrank.tl.transition_matrix` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  kernel = transition_matrix(


  0%|          | 0/406 [00:00<?, ?cell/s]

Setting `softmax_scale=5.3070`


  0%|          | 0/406 [00:00<?, ?cell/s]

    Finish (0:00:02)
Using a connectivity kernel with weight `0.2`
Computing transition matrix based on `adata.obsp['connectivities']`
    Finish (0:00:00)
Computing eigendecomposition of the transition matrix
Adding `adata.uns['eigendecomposition_fwd']`
       `.eigendecomposition`
    Finish (0:00:00)
For 1 macrostate, stationary distribution is computed
Adding `.macrostates`
       `.macrostates_memberships`
       `.coarse_T`
       `.coarse_initial_distribution
       `.coarse_stationary_distribution`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:00)
Adding `adata.obs['terminal_states']`
       `adata.obs['terminal_states_probs']`
       `.terminal_states`
       `.terminal_states_probabilities`
       `.terminal_states_memberships
    Finish`


In [118]:
cr.pl.terminal_states(adata,color='cell_type')
plt.savefig('./adultsscneogermforcellrank_terminal_states0221.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scvelo/plotting/utils.py:869: MatplotlibDeprecationWarning: The draw_all function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use fig.draw_without_rendering() instead.
  cb.draw_all()


In [119]:
cr.tl.initial_states(adata, cluster_key="cell_type")
cr.pl.initial_states(adata, discrete=True)
plt.savefig('./adultsscneogermforcellrank_initial_states0221.pdf')

Accessing `adata.obsp['T_bwd']`
Computing transition matrix based on logits using `'deterministic'` mode
Estimating `softmax_scale` using `'deterministic'` mode


/tmp/ipykernel_167116/2629168253.py:1: DeprecationWarning: `cellrank.tl.initial_states` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.initial_states(adata, cluster_key="cell_type")
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/cellrank/tl/_init_term_states.py:156: DeprecationWarning: `cellrank.tl.transition_matrix` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  kernel = transition_matrix(


  0%|          | 0/406 [00:00<?, ?cell/s]

Setting `softmax_scale=5.3070`


  0%|          | 0/406 [00:00<?, ?cell/s]

    Finish (0:00:01)
Using a connectivity kernel with weight `0.2`
Computing transition matrix based on `adata.obsp['connectivities']`
    Finish (0:00:00)
Computing eigendecomposition of the transition matrix
Adding `adata.uns['eigendecomposition_bwd']`
       `.eigendecomposition`
    Finish (0:00:00)
Computing Schur decomposition
Adding `adata.uns['eigendecomposition_bwd']`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:00)
Computing `2` macrostates
Adding `.macrostates`
       `.macrostates_memberships`
       `.coarse_T`
       `.coarse_initial_distribution
       `.coarse_stationary_distribution`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:00)
Adding `adata.obs['initial_states']`
       `adata.obs['initial_states_probs']`
       `.terminal_states`
       `.terminal_states_probabilities`
       `.terminal_states_memberships
    Finish`


In [120]:
cr.tl.lineages(adata)
cr.pl.lineages(adata, same_plot=False)
plt.savefig('./adultsscneogermforcellrank_fatemaps0221.pdf')

Computing absorption probabilities
Defaulting to `'gmres'` solver.


/tmp/ipykernel_167116/297456376.py:1: DeprecationWarning: `cellrank.tl.lineages` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.lineages(adata)


  0%|          | 0/1 [00:00<?, ?/s]

Adding `adata.obsm['to_terminal_states']`
       `.absorption_probabilities`
    Finish (0:00:00)


/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scvelo/plotting/utils.py:869: MatplotlibDeprecationWarning: The draw_all function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use fig.draw_without_rendering() instead.
  cb.draw_all()


In [121]:
scv.tl.recover_latent_time(
    adata, root_key="initial_states_probs", end_key="terminal_states_probs"
)

computing latent time using initial_states_probs, terminal_states_probs as prior
    finished (0:00:00) --> added 
    'latent_time', shared time (adata.obs)


In [122]:
scv.tl.paga(
    adata,
    groups="cell_type",
    root_key="initial_states_probs",
    end_key="terminal_states_probs",
    use_time_prior="velocity_pseudotime",
)

running PAGA using priors: ['velocity_pseudotime', 'initial_states_probs', 'terminal_states_probs']
    finished (0:00:00) --> added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns)
    'paga/transitions_confidence', velocity transitions (adata.uns)


In [123]:
cr.pl.cluster_fates(adata, mode="bar",cluster_key="cell_type")
plt.savefig('./adultsscneogermforcellrank_barplot_clusterfates0221.pdf')

In [124]:
cr.tl.lineage_drivers(adata)

Adding `adata.varm['terminal_lineage_drivers']`
       `.lineage_drivers`
    Finish (0:00:00)


/tmp/ipykernel_167116/1721556810.py:1: DeprecationWarning: `cellrank.tl.lineage_drivers` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.lineage_drivers(adata)


,SSC_corr,SSC_pval,SSC_qval,SSC_ci_low,SSC_ci_high
PTPRQ,0.484815,2.284681e-26,2.284681e-23,0.406680,0.555909
EEA1,0.474448,3.992478e-25,1.602114e-22,0.395381,0.546535
KANSL1L,0.473438,5.244628e-25,1.748209e-22,0.394282,0.545621
ASH1L,0.469479,1.513882e-24,4.276803e-22,0.389974,0.542036
CCDC122,0.448019,3.650237e-22,8.111637e-20,0.366683,0.522557
...,...,...,...,...,...
RPS8,-0.440017,2.533285e-21,5.066570e-19,-0.515275,-0.358025
RPL28,-0.469019,1.710721e-24,4.276803e-22,-0.541619,-0.389473
RPS7,-0.474436,4.005284e-25,1.602114e-22,-0.546525,-0.395368
RPS18,-0.480742,7.123021e-26,4.748680e-23,-0.552228,-0.402238


In [125]:
cr.pl.lineage_drivers(adata, lineage="SSC", n_genes=16)
plt.savefig('./adultsscneogermforcellrank_SSCdrivergenes0221.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scvelo/plotting/utils.py:869: MatplotlibDeprecationWarning: The draw_all function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use fig.draw_without_rendering() instead.
  cb.draw_all()


In [127]:
# compue DPT, starting from CellRank defined root cell
root_idx = np.where(adata.obs["initial_states"] == "SSC1")[0][0]
adata.uns["iroot"] = root_idx
sc.tl.dpt(adata)

scv.pl.scatter(
    adata,
    color=["cell_type", root_idx, "latent_time", "dpt_pseudotime"],
    fontsize=16,
    cmap="viridis",
    perc=[2, 98],
    colorbar=True,
    dpi=600,
    rescale_color=[0, 1],
    title=["cell_type", "root cell", "latent time", "dpt pseudotime"],
    save='./adultsscneogermforcellrank_SSC1_temporalorderings0221.pdf'
)

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scvelo/plotting/utils.py:869: MatplotlibDeprecationWarning: The draw_all function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use fig.draw_without_rendering() instead.
  cb.draw_all()


saving figure to file ./adultsscneogermforcellrank_SSC1_temporalorderings0221.pdf


In [129]:
cr.tl.terminal_states(
    adata,
    cluster_key="cell_type",
    weight_connectivities=0.2,)
cr.tl.lineages(adata)
model = cr.ul.models.GAM(adata)
cr.tl.lineage_drivers(adata)
cr.pl.heatmap(
    adata,
    model,
    adata.varm['terminal_lineage_drivers']["SSC_corr"].sort_values(ascending=False).index[:100],
    show_absorption_probabilities=True,
    lineages="SSC",
    n_jobs=1,
    backend="loky",
    save='./adultsscneogermforcellrank_SSCgeneexpressiontrendsheatmap0221.pdf'
)

Accessing `adata.obsp['T_fwd']`
Using precomputed transition matrix
Computing eigendecomposition of the transition matrix
Adding `adata.uns['eigendecomposition_fwd']`
       `.eigendecomposition`
    Finish (0:00:00)
For 1 macrostate, stationary distribution is computed
Adding `.macrostates`
       `.macrostates_memberships`
       `.coarse_T`
       `.coarse_initial_distribution
       `.coarse_stationary_distribution`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:00)
Adding `adata.obs['terminal_states']`
       `adata.obs['terminal_states_probs']`
       `.terminal_states`
       `.terminal_states_probabilities`
       `.terminal_states_memberships
    Finish`
Computing absorption probabilities


/tmp/ipykernel_167116/3580534288.py:1: DeprecationWarning: `cellrank.tl.terminal_states` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.terminal_states(
/tmp/ipykernel_167116/3580534288.py:5: DeprecationWarning: `cellrank.tl.lineages` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.lineages(adata)


  0%|          | 0/1 [00:00<?, ?/s]

Adding `adata.obsm['to_terminal_states']`
       `.absorption_probabilities`
    Finish (0:00:00)
Adding `adata.varm['terminal_lineage_drivers']`
       `.lineage_drivers`
    Finish (0:00:00)
Computing trends using `1` core(s)


/tmp/ipykernel_167116/3580534288.py:7: DeprecationWarning: `cellrank.tl.lineage_drivers` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.lineage_drivers(adata)


  0%|          | 0/100 [00:00<?, ?gene/s]

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/pygam/utils.py:649: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bases = (x >= aug_knots[:-1]).astype(np.int) * \
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/pygam/utils.py:650: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

    Finish (0:00:07)
